In [1]:
import scipy

In [2]:
from keras.datasets import mnist
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
from data_loader import DataLoader
import numpy as np
import os

Using TensorFlow backend.


In [1]:
! conda info


     active environment : tf-gpu
    active env location : /home/k_yonhon/anaconda3/envs/tf-gpu
            shell level : 2
       user config file : /home/k_yonhon/.condarc
 populated config files : /home/k_yonhon/.condarc
          conda version : 4.6.8
    conda-build version : 3.17.8
         python version : 3.7.2.final.0
       base environment : /home/k_yonhon/anaconda3  (writable)
           channel URLs : https://repo.anaconda.com/pkgs/main/linux-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/free/linux-64
                          https://repo.anaconda.com/pkgs/free/noarch
                          https://repo.anaconda.com/pkgs/r/linux-64
                          https://repo.anaconda.com/pkgs/r/noarch
                          https://conda.anaconda.org/conda-forge/linux-64
                          https://conda.anaconda.org/conda-forge/noarch
          package cache : /home/k_yonhon/anacon

In [2]:
!python -V

Python 3.6.8 :: Anaconda, Inc.


In [3]:
class Pix2Pix():
    def __init__(self):
        # Input shape
        self.img_rows = 256
        self.img_cols = 256
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        # Configure data loader
        self.dataset_name = 'facades'
        self.data_loader = DataLoader(dataset_name=self.dataset_name,
                                      img_res=(self.img_rows, self.img_cols))
        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.gf = 64
        self.df = 64
        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        #-------------------------
        # Construct Computational
        #   Graph of Generator
        #-------------------------

        # Build the generator
        self.generator = self.build_generator()

        # Input images and their conditioning images
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # By conditioning on B generate a fake version of A
        fake_A = self.generator(img_B)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # Discriminators determines validity of translated images / condition pairs
        valid = self.discriminator([fake_A, img_B])
        self.combined = Model(inputs=[img_A, img_B], outputs=[valid, fake_A])
        self.combined.compile(loss=['mse', 'mae'],
                              loss_weights=[1, 100],
                              optimizer=optimizer)
        
    def build_generator(self):
        """U-Net Generator"""
        def conv2d(layer_input, filters, f_size=4, bn=True):
            """Layers used during downsampling"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            return d
        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            u = BatchNormalization(momentum=0.8)(u)
            u = Concatenate()([u, skip_input])
            return u

        # Image input
        d0 = Input(shape=self.img_shape)

        # Downsampling
        d1 = conv2d(d0, self.gf, bn=False)
        d2 = conv2d(d1, self.gf*2)
        d3 = conv2d(d2, self.gf*4)
        d4 = conv2d(d3, self.gf*8)
        d5 = conv2d(d4, self.gf*8)
        d6 = conv2d(d5, self.gf*8)
        d7 = conv2d(d6, self.gf*8)

        # Upsampling
        u1 = deconv2d(d7, d6, self.gf*8)
        u2 = deconv2d(u1, d5, self.gf*8)
        u3 = deconv2d(u2, d4, self.gf*8)
        u4 = deconv2d(u3, d3, self.gf*4)
        u5 = deconv2d(u4, d2, self.gf*2)
        u6 = deconv2d(u5, d1, self.gf)
        u7 = UpSampling2D(size=2)(u6)
        output_img = Conv2D(self.channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u7)
        return Model(d0, output_img)
    
    def build_discriminator(self):
        def d_layer(layer_input, filters, f_size=4, bn=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            return d
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # Concatenate image and conditioning image by channels to produce input
        combined_imgs = Concatenate(axis=-1)([img_A, img_B])
        d1 = d_layer(combined_imgs, self.df, bn=False)
        d2 = d_layer(d1, self.df*2)
        d3 = d_layer(d2, self.df*4)
        d4 = d_layer(d3, self.df*8)
        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)
        return Model([img_A, img_B], validity)
    
    def train(self, epochs, batch_size=1, sample_interval=50):
        start_time = datetime.datetime.now()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)
        for epoch in range(epochs):
            for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(batch_size)):
                # ---------------------
                #  Train Discriminator
                # ---------------------
                # Condition on B and generate a translated version
                fake_A = self.generator.predict(imgs_B)
                # Train the discriminators (original images = real / generated = Fake)
                d_loss_real = self.discriminator.train_on_batch([imgs_A, imgs_B], valid)
                d_loss_fake = self.discriminator.train_on_batch([fake_A, imgs_B], fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
                # -----------------
                #  Train Generator
                # -----------------
                # Train the generators
                g_loss = self.combined.train_on_batch([imgs_A, imgs_B], [valid, imgs_A])
                elapsed_time = datetime.datetime.now() - start_time
                # Plot the progress
                print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %f] time: %s" % (epoch, epochs,
                                                                        batch_i, self.data_loader.n_batches,
                                                                        d_loss[0], 100*d_loss[1],
                                                                        g_loss[0],
                                                                        elapsed_time))
                # If at save interval => save generated image samples
                if batch_i % sample_interval == 0:
                    self.sample_images(epoch, batch_i)
                    
    def sample_images(self, epoch, batch_i):
        os.makedirs('images/%s' % self.dataset_name, exist_ok=True)
        r, c = 3, 3
        imgs_A, imgs_B = self.data_loader.load_data(batch_size=3, is_testing=True)
        fake_A = self.generator.predict(imgs_B)
        gen_imgs = np.concatenate([imgs_B, fake_A, imgs_A])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        titles = ['Condition', 'Generated', 'Original']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[i])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%s/%d_%d.png" % (self.dataset_name, epoch, batch_i))
        plt.close()

In [ ]:
gan = Pix2Pix()
gan.train(epochs=200, batch_size=1, sample_interval=200)

/home/k_yonhon/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[Epoch 0/200] [Batch 0/400] [D loss: 10.960318, acc:  19%] [G loss: 74.403175] time: 0:00:14.534374
[Epoch 0/200] [Batch 1/400] [D loss: 6.195656, acc:  17%] [G loss: 65.184540] time: 0:00:15.424764
[Epoch 0/200] [Batch 2/400] [D loss: 4.511317, acc:  22%] [G loss: 80.135323] time: 0:00:15.598564
[Epoch 0/200] [Batch 3/400] [D loss: 3.868899, acc:  20%] [G loss: 59.058533] time: 0:00:15.786866
[Epoch 0/200] [Batch 4/400] [D loss: 2.394866, acc:  24%] [G loss: 67.413124] time: 0:00:15.977044
[Epoch 0/200] [Batch 5/400] [D loss: 2.149631, acc:  27%] [G loss: 56.016708] time: 0:00:16.186507
[Epoch 0/200] [Batch 6/400] [D loss: 2.010255, acc:  26%] [G loss: 52.054096] time: 0:00:16.358222
[Epoch 0/200] [Batch 7/400] [D loss: 3.210721, acc:  24%] [G loss: 70.059509] time: 0:00:16.534047
[Epoch 0/200] [Batch 8/400] [D loss: 4.628659, acc:  20%] [G loss: 56.004532] time: 0:00:16.706484
[Epoch 0/200] [Batch 9/400] [D loss: 2.776942, acc:  23%] [G loss: 55.131355] time: 0:00:16.890218
[Epoch 0/

[Epoch 0/200] [Batch 83/400] [D loss: 0.414333, acc:  48%] [G loss: 30.886271] time: 0:00:29.583117
[Epoch 0/200] [Batch 84/400] [D loss: 0.389976, acc:  48%] [G loss: 41.885761] time: 0:00:29.751891
[Epoch 0/200] [Batch 85/400] [D loss: 0.407851, acc:  46%] [G loss: 39.517139] time: 0:00:29.924874
[Epoch 0/200] [Batch 86/400] [D loss: 0.351621, acc:  52%] [G loss: 39.427418] time: 0:00:30.094770
[Epoch 0/200] [Batch 87/400] [D loss: 0.482704, acc:  47%] [G loss: 38.107769] time: 0:00:30.265136
[Epoch 0/200] [Batch 88/400] [D loss: 0.459729, acc:  50%] [G loss: 41.762405] time: 0:00:30.435197
[Epoch 0/200] [Batch 89/400] [D loss: 0.368003, acc:  46%] [G loss: 45.204273] time: 0:00:30.604619
[Epoch 0/200] [Batch 90/400] [D loss: 0.384531, acc:  49%] [G loss: 50.492416] time: 0:00:30.784550
[Epoch 0/200] [Batch 91/400] [D loss: 0.367446, acc:  49%] [G loss: 39.511421] time: 0:00:30.955274
[Epoch 0/200] [Batch 92/400] [D loss: 0.463284, acc:  46%] [G loss: 29.901133] time: 0:00:31.123992


[Epoch 0/200] [Batch 165/400] [D loss: 0.348435, acc:  45%] [G loss: 32.810516] time: 0:00:43.596152
[Epoch 0/200] [Batch 166/400] [D loss: 0.252041, acc:  62%] [G loss: 58.310913] time: 0:00:43.764841
[Epoch 0/200] [Batch 167/400] [D loss: 0.248599, acc:  60%] [G loss: 40.336697] time: 0:00:43.936449
[Epoch 0/200] [Batch 168/400] [D loss: 0.407980, acc:  40%] [G loss: 37.726662] time: 0:00:44.106881
[Epoch 0/200] [Batch 169/400] [D loss: 0.399842, acc:  45%] [G loss: 35.677902] time: 0:00:44.277383
[Epoch 0/200] [Batch 170/400] [D loss: 0.378832, acc:  50%] [G loss: 37.538334] time: 0:00:44.446290
[Epoch 0/200] [Batch 171/400] [D loss: 1.102813, acc:  27%] [G loss: 52.611706] time: 0:00:44.616661
[Epoch 0/200] [Batch 172/400] [D loss: 0.647994, acc:  45%] [G loss: 40.929535] time: 0:00:44.785761
[Epoch 0/200] [Batch 173/400] [D loss: 0.383664, acc:  56%] [G loss: 39.436329] time: 0:00:44.955986
[Epoch 0/200] [Batch 174/400] [D loss: 0.367208, acc:  56%] [G loss: 52.330570] time: 0:00:

[Epoch 0/200] [Batch 247/400] [D loss: 0.277499, acc:  61%] [G loss: 38.558743] time: 0:00:57.839929
[Epoch 0/200] [Batch 248/400] [D loss: 0.267240, acc:  52%] [G loss: 31.128633] time: 0:00:58.008166
[Epoch 0/200] [Batch 249/400] [D loss: 0.390037, acc:  43%] [G loss: 27.333736] time: 0:00:58.178734
[Epoch 0/200] [Batch 250/400] [D loss: 0.272490, acc:  60%] [G loss: 40.968571] time: 0:00:58.349790
[Epoch 0/200] [Batch 251/400] [D loss: 0.275404, acc:  63%] [G loss: 44.017517] time: 0:00:58.520425
[Epoch 0/200] [Batch 252/400] [D loss: 0.347411, acc:  50%] [G loss: 35.103981] time: 0:00:58.690941
[Epoch 0/200] [Batch 253/400] [D loss: 0.340623, acc:  57%] [G loss: 35.063763] time: 0:00:58.864321
[Epoch 0/200] [Batch 254/400] [D loss: 0.308312, acc:  53%] [G loss: 29.293510] time: 0:00:59.032924
[Epoch 0/200] [Batch 255/400] [D loss: 0.263368, acc:  63%] [G loss: 46.566895] time: 0:00:59.204894
[Epoch 0/200] [Batch 256/400] [D loss: 0.354604, acc:  49%] [G loss: 29.058393] time: 0:00:

...

[Epoch 4/200] [Batch 377/400] [D loss: 0.050612, acc:  97%] [G loss: 38.319695] time: 0:06:01.789293
[Epoch 4/200] [Batch 378/400] [D loss: 0.070998, acc:  95%] [G loss: 35.598381] time: 0:06:01.963389
[Epoch 4/200] [Batch 379/400] [D loss: 0.031695, acc:  97%] [G loss: 31.928898] time: 0:06:02.138495
[Epoch 4/200] [Batch 380/400] [D loss: 0.018308, acc: 100%] [G loss: 43.880463] time: 0:06:02.314415
[Epoch 4/200] [Batch 381/400] [D loss: 0.010929, acc: 100%] [G loss: 36.793179] time: 0:06:02.488789
[Epoch 4/200] [Batch 382/400] [D loss: 0.006644, acc: 100%] [G loss: 35.157913] time: 0:06:02.663743
[Epoch 4/200] [Batch 383/400] [D loss: 0.017279, acc: 100%] [G loss: 33.597095] time: 0:06:02.836506
[Epoch 4/200] [Batch 384/400] [D loss: 0.019884, acc:  99%] [G loss: 21.995352] time: 0:06:03.011980
[Epoch 4/200] [Batch 385/400] [D loss: 0.030258, acc: 100%] [G loss: 36.961227] time: 0:06:03.186544
[Epoch 4/200] [Batch 386/400] [D loss: 0.077604, acc:  91%] [G loss: 26.812397] time: 0:06:

[Epoch 5/200] [Batch 61/400] [D loss: 0.014451, acc: 100%] [G loss: 33.295864] time: 0:06:16.581230
[Epoch 5/200] [Batch 62/400] [D loss: 0.076640, acc:  92%] [G loss: 40.724770] time: 0:06:16.754367
[Epoch 5/200] [Batch 63/400] [D loss: 0.068236, acc:  90%] [G loss: 34.444851] time: 0:06:16.929290
[Epoch 5/200] [Batch 64/400] [D loss: 0.065163, acc:  94%] [G loss: 29.497665] time: 0:06:17.104279
[Epoch 5/200] [Batch 65/400] [D loss: 0.026919, acc:  99%] [G loss: 44.244396] time: 0:06:17.279065
[Epoch 5/200] [Batch 66/400] [D loss: 0.013712, acc: 100%] [G loss: 33.646854] time: 0:06:17.453666
[Epoch 5/200] [Batch 67/400] [D loss: 0.039205, acc:  96%] [G loss: 36.903175] time: 0:06:17.630474
[Epoch 5/200] [Batch 68/400] [D loss: 0.043613, acc:  97%] [G loss: 29.279388] time: 0:06:17.806346
[Epoch 5/200] [Batch 69/400] [D loss: 0.034147, acc:  99%] [G loss: 35.214359] time: 0:06:17.981500
[Epoch 5/200] [Batch 70/400] [D loss: 0.030125, acc:  99%] [G loss: 41.881695] time: 0:06:18.155671


[Epoch 5/200] [Batch 143/400] [D loss: 0.048119, acc:  96%] [G loss: 34.962337] time: 0:06:30.930035
[Epoch 5/200] [Batch 144/400] [D loss: 0.021739, acc:  99%] [G loss: 30.685255] time: 0:06:31.104657
[Epoch 5/200] [Batch 145/400] [D loss: 0.021334, acc:  99%] [G loss: 33.171398] time: 0:06:31.279301
[Epoch 5/200] [Batch 146/400] [D loss: 0.021438, acc:  98%] [G loss: 40.235859] time: 0:06:31.453290
[Epoch 5/200] [Batch 147/400] [D loss: 0.016554, acc: 100%] [G loss: 32.610481] time: 0:06:31.629441
[Epoch 5/200] [Batch 148/400] [D loss: 0.010703, acc: 100%] [G loss: 28.607239] time: 0:06:31.803991
[Epoch 5/200] [Batch 149/400] [D loss: 0.014240, acc:  99%] [G loss: 34.883369] time: 0:06:31.983653
[Epoch 5/200] [Batch 150/400] [D loss: 0.038468, acc:  98%] [G loss: 25.757053] time: 0:06:32.161008
[Epoch 5/200] [Batch 151/400] [D loss: 0.025080, acc:  99%] [G loss: 39.351448] time: 0:06:32.336901
[Epoch 5/200] [Batch 152/400] [D loss: 0.025062, acc: 100%] [G loss: 42.012241] time: 0:06:

[Epoch 5/200] [Batch 225/400] [D loss: 0.009675, acc: 100%] [G loss: 46.293339] time: 0:06:45.549573
[Epoch 5/200] [Batch 226/400] [D loss: 0.029319, acc:  99%] [G loss: 28.905943] time: 0:06:45.726533
[Epoch 5/200] [Batch 227/400] [D loss: 0.051500, acc:  96%] [G loss: 35.268845] time: 0:06:45.902813
[Epoch 5/200] [Batch 228/400] [D loss: 0.024727, acc:  97%] [G loss: 29.794050] time: 0:06:46.078301
[Epoch 5/200] [Batch 229/400] [D loss: 0.044778, acc:  97%] [G loss: 39.136871] time: 0:06:46.253569
[Epoch 5/200] [Batch 230/400] [D loss: 0.024947, acc:  99%] [G loss: 34.249336] time: 0:06:46.430317
[Epoch 5/200] [Batch 231/400] [D loss: 0.021432, acc:  98%] [G loss: 35.208607] time: 0:06:46.607626
[Epoch 5/200] [Batch 232/400] [D loss: 0.031728, acc:  99%] [G loss: 43.999718] time: 0:06:46.783269
[Epoch 5/200] [Batch 233/400] [D loss: 0.013604, acc: 100%] [G loss: 33.110645] time: 0:06:46.959689
[Epoch 5/200] [Batch 234/400] [D loss: 0.014551, acc: 100%] [G loss: 37.040665] time: 0:06:

[Epoch 5/200] [Batch 307/400] [D loss: 0.024377, acc:  99%] [G loss: 34.721821] time: 0:06:59.899083
[Epoch 5/200] [Batch 308/400] [D loss: 0.150383, acc:  79%] [G loss: 35.652695] time: 0:07:00.073046
[Epoch 5/200] [Batch 309/400] [D loss: 0.051289, acc:  94%] [G loss: 38.717915] time: 0:07:00.248231
[Epoch 5/200] [Batch 310/400] [D loss: 0.050008, acc:  95%] [G loss: 35.257141] time: 0:07:00.422824
[Epoch 5/200] [Batch 311/400] [D loss: 0.054829, acc:  97%] [G loss: 32.347416] time: 0:07:00.598261
[Epoch 5/200] [Batch 312/400] [D loss: 0.074078, acc:  90%] [G loss: 37.233677] time: 0:07:00.771971
[Epoch 5/200] [Batch 313/400] [D loss: 0.035121, acc:  97%] [G loss: 28.652719] time: 0:07:00.946316
[Epoch 5/200] [Batch 314/400] [D loss: 0.048477, acc:  96%] [G loss: 46.342884] time: 0:07:01.119413
[Epoch 5/200] [Batch 315/400] [D loss: 0.065845, acc:  92%] [G loss: 28.890001] time: 0:07:01.294132
[Epoch 5/200] [Batch 316/400] [D loss: 0.129325, acc:  88%] [G loss: 37.647472] time: 0:07: